# Visualizing HRRR Model Data Over a General Area Using Herbie
In this notebook we will download and plot HRRR wind data for the wind profiler and sodar sites.

In [ ]:
#Import Modules
from herbie import Herbie
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units

### Downloading the Data
Here you can input the date and pressure level and get the full 48-hour HRRR forecast for that day.

In [ ]:
yr=input('Year (yyyy):')
m=input('Month (mm):')
d=input('Day (dd):')
date=(f'{yr}-{m}-{d}')
level = input(f'Enter level (surface, 1000 mb, 925 mb, 850 mb, etc.):')
H=[]
ds=[]
for a in range(0, 49):
    H.append(Herbie(f"{yr}-{m}-{d}",  # model run date
        model="hrrr",  # model name
        product="prs", #product
        fxx=a,  # forecast lead time
              ))
    ds.append(H[a].xarray(f":{level}:", remove_grib=False))
    if level == str('surface'):
        ds.append(ds[a][0])

# Plot the Data
Here we will plot the data for the area surrounding the three sites, which are denoted by the black stars.

In [ ]:
#Coordinates:
#RI: 41.4456, -71.4357
#CC: 42.03, -70.049
#Sodar: 41.2453, -70.105
for a in range(0, 49):
    plt.figure(1, figsize=(15, 15))
    ax = plt.subplot(111, projection=ccrs.PlateCarree())
    ax.set_extent([-69.5, -72, 42.5, 41]), ccrs.PlateCarree() 
    cf = ax.contourf(ds[0].longitude, ds[0].latitude, 
                     (mpcalc.wind_speed(ds[a].u.values*units.meter/units.second, 
                                        ds[a].v.values*units.meter/units.second)).m, 
                     np.arange(-5, 25, 0.5), cmap=plt.cm.BuPu)
    plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeature.STATES.with_scale('50m'))
    ax.scatter(-71.4357, 41.4456, 400, marker='*', color='black', transform=ccrs.PlateCarree())
    ax.scatter(-70.049, 42.03, 400, marker='*', color='black', transform=ccrs.PlateCarree())
    ax.scatter(-70.105, 41.2453, 400, marker='*', color='black', transform=ccrs.PlateCarree())
    plt.title(f'Time: {date} FH {a}   HRRR Wind Speed at {level}')
    plt.show()

In [ ]:
# I want to find the speed at a certain lat/lon point.
lat = [41.4456, 42.03, 41.2453]
lon = [-71.4357, -70.049, -70.105]
point_ds=[]
for a in range(0, 25):
    ds[a]['longitude'] = (ds[a]['longitude'] + 180) % 360 - 180
    for b in range(0, 3):
        # First, find the index of the grid point nearest a specific lat/lon.   
        abslat = np.abs(ds[a].latitude-lat[b])
        abslon = np.abs(ds[a].longitude-lon[b])
        c = np.maximum(abslon, abslat)
        ([yloc], [xloc]) = np.where(c == np.min(c))
        # Now I can use that index location to get the values at the x/y diminsion
        point_ds.append(ds[a].sel(x=xloc, y=yloc))

Next we calculate the wind speed and direction for each site using the u and v wind components.

In [ ]:
rhod_spd=[]
caco_spd=[]
sodar_spd=[]
rhod_dir=[]
caco_dir=[]
sodar_dir=[]
times=[]
for d in range(0, 75, 3):
    rhod_spd.append((mpcalc.wind_speed(point_ds[d].u.values * units.meter/units.second, point_ds[d].v.values * units.meter/units.second)).m)
    caco_spd.append((mpcalc.wind_speed(point_ds[d+1].u.values * units.meter/units.second, point_ds[d+1].v.values * units.meter/units.second)).m)
    sodar_spd.append((mpcalc.wind_speed(point_ds[d+2].u.values * units.meter/units.second, point_ds[d+2].v.values * units.meter/units.second)).m)
    rhod_dir.append((mpcalc.wind_direction(point_ds[d].u.values * units.meter/units.second, point_ds[d].v.values * units.meter/units.second)).m)
    caco_dir.append((mpcalc.wind_direction(point_ds[d+1].u.values * units.meter/units.second, point_ds[d+1].v.values * units.meter/units.second)).m)
    sodar_dir.append((mpcalc.wind_direction(point_ds[d+2].u.values * units.meter/units.second, point_ds[d+2].v.values * units.meter/units.second)).m)
    times.append(point_ds[d].valid_time.values)

In [ ]:
plt.figure(figsize=(15, 15))
ax = plt.subplot(311)
ax.plot(times, rhod_spd)
ax.set_title(f'HRRR {level} Wind Speed at Rhode Island RWP site')
ax.set_ylabel('Speed (m/s)')
ax.set_xlabel('Time')
ax = plt.subplot(312)
ax.plot(times, caco_spd)
ax.set_title(f'HRRR {level} Wind Speed at Cape Cod RWP site')
ax.set_ylabel('Speed (m/s)')
ax.set_xlabel('Time')
ax = plt.subplot(313)
ax.plot(times, sodar_spd)
ax.set_title(f'HRRR {level} Wind Speed at SODAR site')
ax.set_ylabel('Speed (m/s)')
ax.set_xlabel('Time')

In [ ]:
plt.figure(figsize=(15, 15))
ax = plt.subplot(311)
ax.plot(times, rhod_dir)
ax.set_title(f'HRRR {level} Wind Direction at Rhode Island RWP site')
ax.set_ylabel('Direction (deg)')
ax.set_xlabel('Time')
ax = plt.subplot(312)
ax.plot(times, caco_dir)
ax.set_title(f'HRRR {level} Wind Direction at Cape Cod RWP site')
ax.set_ylabel('Direction (deg)')
ax.set_xlabel('Time')
ax = plt.subplot(313)
ax.plot(times, sodar_dir)
ax.set_title(f'HRRR {level} Wind Direction at SODAR site')
ax.set_ylabel('Direction (deg)')
ax.set_xlabel('Time')